## install

In [1]:
!pip install turicreate

     |████████████████████████████████| 92.0MB 1.2MB/s eta 0:00:01Requirement already satisfied: requests>=2.9.1 in /usr/local/lib/python3.7/dist-packages (from turicreate) (2.23.0)
     |████████████████████████████████| 327kB 35.6MB/s 
     |████████████████████████████████| 86.4MB 48kB/s 
     |████████████████████████████████| 3.6MB 32.5MB/s 
     |████████████████████████████████| 3.5MB 27.5MB/s 
     |████████████████████████████████| 51kB 5.9MB/s 
     |████████████████████████████████| 3.8MB 26.2MB/s 
     |████████████████████████████████| 450kB 32.5MB/s 
     |████████████████████████████████| 2.9MB 24.7MB/s 
     |████████████████████████████████| 18.3MB 172kB/s 
  Created wheel for resampy: filename=resampy-0.2.1-cp37-none-any.whl size=320858 sha256=9bd30be8338b765f34ec025c3f2838fd524826e1e587ff39ddb74e41866a7de3
  Stored in directory: /root/.cache/pip/wheels/ff/4f/ed/2e6c676c23efe5394bb40ade50662e90eb46e29b48324c5f9b
  Created wheel for prettytable: filename=prettytable-0.

## imports

In [2]:
import json
import numpy as np
import pandas as pd
import turicreate as tc
from PIL import Image
from numpy import asarray
from tensorflow import keras
import os
from collections import Counter

## kaggle data API

In [3]:
# Installing the Kaggle package
!pip install kaggle 
!mkdir /root/.kaggle/
!mkdir ./datasets

#Important Note: complete this with your own key - after running this for the first time remmember to **remove** your API_KEY
api_token = {"username":"idoefroni","key":"fc35681504a8107575c7a50c67f504cc"}

# creating kaggle.json file with the personal API-Key details 
# You can also put this file on your Google Drive
with open('/root/.kaggle/kaggle.json', 'w') as file:
  json.dump(api_token, file)
!chmod 600 /root/.kaggle/kaggle.json

## cast_img

this function receive dataframe with 2 columns (path,code) and return a two list of the reformeted image and the label 

In [4]:
def cast_img(list_img_path):
  list_rgb = []
  basewidth = 32
  # load the image and convert into numpy array
  for row in list_img_path:
    img = Image.open(row)
    #resize
    img = img.resize((basewidth,basewidth), Image.ANTIALIAS)
    numpydata = asarray(img)
    list_rgb.append(numpydata)
  return list_rgb

In [5]:
def balance_data(data_unbalanced):
  g = data_unbalanced.groupby('label')
  data_unbalanced = g.apply(lambda x: x.sample(g.size().min()).reset_index(drop=True))
  data_unbalanced = data_unbalanced.reset_index(drop=True)
  return data_unbalanced

## get_monkey_data

In [6]:
def get_monkey_data():
  # monky dataset load and pre process 
  !mkdir ./datasets/10-monkey-species
  # download the dataset from Kaggle and unzip it
  !kaggle datasets download slothkong/10-monkey-species -p ./datasets/10-monkey-species
  !unzip ./datasets/10-monkey-species/*.zip  -d ./datasets/10-monkey-species/ 

  DATASET_PATH = "./datasets/10-monkey-species/"
  train_data = tc.image_analysis.load_images(f"{DATASET_PATH}/training/", with_path=True)
  validate_data = tc.image_analysis.load_images(f"{DATASET_PATH}/validation/", with_path=True)
  train_data.materialize()
  validate_data.materialize()
  print(len(train_data))
  print(len(validate_data))

  merge_data = train_data.join(validate_data, how='outer')
  merge_data.materialize()
  merge_data['label'] = merge_data['path'].apply(lambda p: p.split('/')[-2])
  merge_data.materialize()
  print(len(merge_data))

  #need to change time duo to complexty
  df_all_data = merge_data.to_dataframe()
  
  df_all_data = balance_data(df_all_data)

  #need to change time duo to complexty
  df_all_data.label = pd.Categorical(df_all_data.label)
  df_all_data['code'] = df_all_data.label.cat.codes
  df_all_data = df_all_data.drop(columns=['label'])
  df_all_data = df_all_data.rename(columns={"code": "label"})

  #call to cast img function
  list_img = cast_img(list(df_all_data['path']))

  df_all_data = df_all_data.drop(columns=['path'])
  df_all_data = df_all_data.drop(columns=['image'])

  arr = np.array(list_img)
  list_img = arr.tolist()
  df_all_data['data'] = list_img

  #reduce data

  df_all_data = df_all_data.iloc[::2]
  print(Counter(list(df_all_data['label'])))
  print(len(df_all_data))
  df_all_data.to_csv("monkey.csv")

## get_fruits_data

In [7]:
def get_fruits_data():
  print('fruits')
  # monky dataset load and pre process 
  !mkdir ./datasets/360-fruits
  # download the dataset from Kaggle and unzip it

  !kaggle datasets download -d moltean/fruits -p ./datasets/360-fruits
  !unzip ./datasets/360-fruits/*.zip  -d ./datasets/360-fruits/ 
  DATASET_PATH = "./datasets/360-fruits/fruits-360/"
  train_data = tc.image_analysis.load_images(f"{DATASET_PATH}/Training/", with_path=True)
  train_data['label'] = train_data['path'].apply(lambda p: p.split('/')[-2])
  test_data = tc.image_analysis.load_images(f"{DATASET_PATH}/Test/", with_path=True)
  test_data['label'] = test_data['path'].apply(lambda p: p.split('/')[-2])

  train_data.materialize()
  test_data.materialize()
  print(len(train_data) + len(test_data))

  merge_data = train_data.join(test_data, how='outer')
  merge_data.materialize()
  print(len(merge_data))

  #need to change time duo to complexty
  df_all_data = merge_data.to_dataframe()

  df_all_data = balance_data(df_all_data)

  #need to change time duo to complexty
  df_all_data.label = pd.Categorical(df_all_data.label)
  df_all_data['code'] = df_all_data.label.cat.codes
  df_all_data = df_all_data.drop(columns=['label'])
  df_all_data = df_all_data.rename(columns={"code": "label"})

  #call to cast img function
  list_img = cast_img(list(df_all_data['path']))

  df_all_data = df_all_data.drop(columns=['path'])
  df_all_data = df_all_data.drop(columns=['image'])

  arr = np.array(list_img)
  list_img = arr.tolist()

  df_all_data['data'] = list_img
  df_all_data = df_all_data.iloc[::3]
  print(Counter(list(df_all_data['label'])))

  list_range = list(range(30))
  print(list_range)
  df1 = df_all_data[df_all_data['label'].isin(list_range)]
  print('fruits1')
  print(Counter(list(df1['label'])))
  df1.to_csv("fruits1New.csv")

  list_range = list(range(30,50))
  print(list_range)
  df2 = df_all_data[df_all_data['label'].isin(list_range)]
  print('fruits2')
  print(Counter(list(df2['label'])))
  df2.to_csv("fruits2New.csv")

  list_range = list(range(50,65))
  print(list_range)
  df3 = df_all_data[df_all_data['label'].isin(list_range)]
  print('fruits3')
  print(Counter(list(df3['label'])))
  df3.to_csv("fruits3New.csv")

  list_range = list(range(65,75))
  print(list_range)
  df4 = df_all_data[df_all_data['label'].isin(list_range)]
  print('fruits4')
  print(Counter(list(df4['label'])))
  df4.to_csv("fruits4New.csv")

  # df_all_data.to_csv("fruits.csv")

## get_flowers_data

In [8]:
def get_flowers_data():
  print('flowers')
  #dataset load and pre process 
  !mkdir ./datasets/flowers
  # download the dataset from Kaggle and unzip it

  !kaggle datasets download -d alxmamaev/flowers-recognition -p ./datasets/flowers
  !unzip ./datasets/flowers/*.zip  -d ./datasets/flowers/ 
  DATASET_PATH = "./datasets/flowers/"

  all_data = tc.image_analysis.load_images(f"{DATASET_PATH}/", with_path=True)
  all_data['label'] = all_data['path'].apply(lambda p: p.split('/')[-2])

  all_data.materialize()
  print(all_data)

  df_all_data = all_data.to_dataframe()
  df_all_data = balance_data(df_all_data)

  #need to change time duo to complexty
  df_all_data.label = pd.Categorical(df_all_data.label)
  df_all_data['code'] = df_all_data.label.cat.codes
  df_all_data = df_all_data.drop(columns=['label'])
  df_all_data = df_all_data.rename(columns={"code": "label"})

  #call to cast img function
  list_img = cast_img(list(df_all_data['path']))

  df_all_data = df_all_data.drop(columns=['path'])
  df_all_data = df_all_data.drop(columns=['image'])

  arr = np.array(list_img)
  list_img = arr.tolist()

  df_all_data['data'] = list_img
  print(len(df_all_data))

  df_all_data = df_all_data.iloc[::10]
  print(Counter(list(df_all_data['label'])))

  df_all_data.to_csv("flowers.csv")

## get_butterfly_data

In [9]:
def get_butterfly_data():
  print("butterfly")
  #dataset load and pre process 
  !mkdir ./datasets/butterfly
  # download the dataset from Kaggle and unzip it

  !kaggle datasets download -d gpiosenka/butterfly-images40-species -p ./datasets/butterfly
  !unzip ./datasets/butterfly/*.zip  -d ./datasets/butterfly/ 
  DATASET_PATH = "./datasets/butterfly/butterflies/"
  train_data = tc.image_analysis.load_images(f"{DATASET_PATH}/train/", with_path=True)
  train_data['label'] = train_data['path'].apply(lambda p: p.split('/')[-2])
  test_data = tc.image_analysis.load_images(f"{DATASET_PATH}/test/", with_path=True)
  test_data['label'] = test_data['path'].apply(lambda p: p.split('/')[-2])
  valid_data = tc.image_analysis.load_images(f"{DATASET_PATH}/valid/", with_path=True)
  valid_data['label'] = valid_data['path'].apply(lambda p: p.split('/')[-2])

  
  train_data.materialize()
  test_data.materialize()
  valid_data.materialize()
  print(len(train_data) + len(test_data) + len(valid_data))

  merge_data = train_data.join(test_data, how='outer')
  merge_data.materialize()
  merge_data = merge_data.join(valid_data, how='outer')
  merge_data.materialize()
  print(len(merge_data))


  #need to change time duo to complexty
  df_all_data = merge_data.to_dataframe()

  df_all_data = balance_data(df_all_data)

  #need to change time duo to complexty
  df_all_data.label = pd.Categorical(df_all_data.label)
  df_all_data['code'] = df_all_data.label.cat.codes
  df_all_data = df_all_data.drop(columns=['label'])
  df_all_data = df_all_data.rename(columns={"code": "label"})

  #call to cast img function
  list_img = cast_img(list(df_all_data['path']))

  df_all_data = df_all_data.drop(columns=['path'])
  df_all_data = df_all_data.drop(columns=['image'])

  arr = np.array(list_img)
  list_img = arr.tolist()

  df_all_data['data'] = list_img
  print(Counter(list(df_all_data['label'])))

  list_range = [0,1,2,3,4]
  df1 = df_all_data[df_all_data['label'].isin(list_range)]
  print('butterfly1')
  print(Counter(list(df1['label'])))
  print(len(df1))
  df1.to_csv("butterfly1.csv")

  list_range = [5,15,20,25]
  df2 = df_all_data[df_all_data['label'].isin(list_range)]
  print('butterfly2')
  print(Counter(list(df2['label'])))
  print(len(df2))
  df2.to_csv("butterfly2.csv")

  list_range = [30,40,49]
  df3 = df_all_data[df_all_data['label'].isin(list_range)]
  print('butterfly3')
  print(Counter(list(df3['label'])))
  print(len(df3))
  df3.to_csv("butterfly3.csv")

  df_all_data.to_csv("butterfly.csv")

## get_intel_data


In [10]:
def get_intel_data():
  print("intel")
  #dataset load and pre process 
  !mkdir ./datasets/intel
  # download the dataset from Kaggle and unzip it

  !kaggle datasets download -d puneet6060/intel-image-classification -p ./datasets/intel
  !unzip ./datasets/intel/*.zip  -d ./datasets/intel/ 
  DATASET_PATH = "./datasets/intel/"
  train_data = tc.image_analysis.load_images(f"{DATASET_PATH}/seg_train/seg_train/", with_path=True)
  train_data['label'] = train_data['path'].apply(lambda p: p.split('/')[-2])
  test_data = tc.image_analysis.load_images(f"{DATASET_PATH}/seg_test/seg_test/", with_path=True)
  test_data['label'] = test_data['path'].apply(lambda p: p.split('/')[-2])

  train_data.materialize()
  test_data.materialize()

  merge_data = train_data.join(test_data, how='outer')
  merge_data.materialize()

  #need to change time duo to complexty
  df_all_data = merge_data.to_dataframe()

  df_all_data = balance_data(df_all_data)

  #need to change time duo to complexty
  df_all_data.label = pd.Categorical(df_all_data.label)
  df_all_data['code'] = df_all_data.label.cat.codes
  df_all_data = df_all_data.drop(columns=['label'])
  df_all_data = df_all_data.rename(columns={"code": "label"})

  #call to cast img function
  list_img = cast_img(list(df_all_data['path']))

  df_all_data = df_all_data.drop(columns=['path'])
  df_all_data = df_all_data.drop(columns=['image'])

  arr = np.array(list_img)
  list_img = arr.tolist()

  df_all_data['data'] = list_img

  df_all_data = df_all_data.iloc[::10]
  df_all_data = df_all_data.iloc[::2] 
  print(Counter(list(df_all_data['label'])))

  df_all_data.to_csv("intel.csv")

In [11]:
def data_csv_kaggle(flag):
  if flag == 'monkey':
    get_monkey_data()
  if flag == 'fruits':
    get_fruits_data()
  if flag == 'flowers':
    get_flowers_data()
  if flag == 'butterfly':
    get_butterfly_data()
  if flag == 'intel':
    get_intel_data()

In [ ]:
list_data = ['monkey','fruits','flowers','butterfly','intel']
for name in list_data:
  data_csv_kaggle(name)